In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

['575-cosine-similarity', 'df-all-tokenized', 'csc-575-final-project-common-numbers']


In [2]:
df= pd.read_csv('../input/csc-575-final-project-common-numbers/df_all_stemmed_cos_sim_num.csv')

In [3]:
df

,attribute,id,product_description,product_title,product_uid,relevance,search_term,product_description_tokens,product_title_tokens,search_term_tokens,attribute_tokens,product_description_stemmed,product_title_stemmed,search_term_stemmed,attribute_stemmed,cos_description,cos_title,cos_attribute,cos_description_stemmed,cos_title_stemmed,cos_attribute_stemmed,num_in_description,num_in_title,num_in_attribute
0,Versatile connector for various 90 connections...,2,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,angle bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,angl bracket,versatil connector for variou connect and home...,0,0,0,0.148522,0.408248,0.081111,NaN,NaN,NaN
1,Versatile connector for various 90 connections...,3,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,not only do angles make joints stronger they a...,simpson strongtie 12gauge angle,l bracket,versatile connector for various 90 connections...,not onli do angl make joint stronger they also...,simpson strongti angl,l bracket,versatil connector for variou connect and home...,0,0,0,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,"Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...",9,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,behr premium textured deckover is an innovativ...,behr premium textured deckover 1gal sc141 tugb...,deck over,brushrollerspray 663 in 776 in 663 in revives ...,behr premium textur deckov is an innov solid c...,behr premium textur deckov tugboat wood and co...,deck over,brushrollerspray in in in reviv wood and compo...,0,0,0,0.112906,0.000000,0.080845,NaN,NaN,NaN
3,Combo Tub and Shower No Includes the trim kit ...,16,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,rain shower head,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,rain shower head,combo tub and shower no includ the trim kit on...,0,0,0,0.041451,0.166667,0.096225,NaN,NaN,NaN
4,Combo Tub and Shower No Includes the trim kit ...,17,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,update your bathroom with the delta vero singl...,delta vero 1handle shower only faucet trim kit...,shower only faucet,combo tub and shower no includes the trim kit ...,updat your bathroom with the delta vero single...,delta vero shower onli faucet trim kit in chro...,shower onli faucet,combo tub and shower no includ the trim kit on...,0,0,0,0.124354,0.500000,0.144338,NaN,NaN,NaN
5,Over the Range Microwave 18.5 in 17.13 in 29.9...,18,Achieving delicious results is almost effortle...,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convection otr,achieving delicious results is almost effortle...,whirlpool 19 cu ft over the range convection m...,convection otr,over the range microwave 185 in 1713 in 2994 i...,achiev delici result is almost effortless with...,whirlpool cu ft over the rang convect microwav...,convect otr,over the rang microwav in in in spaciou cu ft ...,0,0,0,0.054831,0.188982,0.034805,NaN,NaN,NaN
6,Over the Range Microwave 18.5 in 17.13 in 29.9...,20,Achieving delicious results is almost effortle...,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove,achieving delicious results is almost effortle...,whirlpool 19 cu ft over the range convection m...,microwave over stove,over the range microwave 185 in 1713 in 2994 i...,achiev delici result is almost e

In [4]:
# df_train = df.iloc[:74067]
# df_test = df.iloc[74067:]

In [5]:
# df_selected = df_test[['product_description_tokens','product_title_tokens','attribute_tokens','search_term_tokens','num_in_description','num_in_title','num_in_attribute']]
# df_selected

In [6]:
def get_jaccard_sim(str1, str2): 
    try:
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    except:
        return np.nan

In [7]:
# if search term doesnt contain numbers ==> -1
# if any number in search term matches decriptions ==> 1
# if search term contains numbers, but numbers dont match descriptions ==>0
def common_num(str1,str2):
    try:
        count_common = np.nan
        for word in str1.split():
            if bool(re.match('[0-9]', word)) == True or bool(re.match('[a-z][0-9]', word)) == True or word.isdigit():
                count_common = 0
                if str2.find(word)>=0:
                    count_common+=1
                else:
                    count_common+= 0

        if count_common == 0:
            return ('no common number')
        elif count_common>0:
            return ('common number')
    except:
        return np.nan

In [8]:
df['num_in_title'] = df[['search_term_tokens','product_title_tokens']].apply(lambda x: common_num(*x),axis = 1)
df['num_in_description'] = df[['search_term_tokens','product_description_tokens']].apply(lambda x: common_num(*x),axis = 1)
df['num_in_attribute'] = df[['search_term_tokens','attribute_tokens']].apply(lambda x: common_num(*x),axis = 1)


In [9]:
df['jc_title'] = df[['search_term_tokens','product_title_tokens']].apply(lambda x: get_jaccard_sim(*x),axis = 1)
df['jc_description'] = df[['search_term_tokens','product_description_tokens']].apply(lambda x: get_jaccard_sim(*x),axis = 1)
df['jc_attribute'] = df[['search_term_tokens','attribute_tokens']].apply(lambda x: get_jaccard_sim(*x),axis = 1)

In [10]:
df.to_csv('df_all_stemmed_cos_sim_num_jc.csv',index = False)